In [1]:
import pandas as pd 
import numpy as np
from xgboost import XGBClassifier

In [2]:
train = pd.read_csv('../data/train.csv')
valid = pd.read_csv('../data/validation.csv')
test = pd.read_csv('../data/test.csv')
col = pd.read_csv('../data/columns.csv', encoding='cp949')

In [3]:
print(train.columns)
print(valid.columns)
print(test.columns)
print(col.columns)

Index(['rid', 'id', 'set', 'age', 'sex', 'HTN', 'DM', 'plt', 'tpro', 'alb',
       'chol', 'bun', 'cr', 'eGFR', 'blood_dip', 'protein_dip', 'acr_urine',
       'pcr_urine', 'glu', 'glu_urine', 'esterase_urine', 'dysmorphism', 'RBC',
       'WBC', 'RBC_re', 'WBC_re', 'RBC_binary', 'WBC_binary',
       'esterase_binary', 'Biopsy_HR', 'Biopsy_HW', 'Biopsy_HK', 'hr_A1',
       'hr_proba', 'hw_A2', 'hw_proba', 'hk_A3', 'hk_proba'],
      dtype='object')
Index(['rid', 'id', 'set', 'age', 'sex', 'HTN', 'DM', 'plt', 'tpro', 'alb',
       'chol', 'bun', 'cr', 'eGFR', 'blood_dip', 'protein_dip', 'acr_urine',
       'pcr_urine', 'glu', 'glu_urine', 'esterase_urine', 'dysmorphism', 'RBC',
       'WBC', 'RBC_re', 'WBC_re', 'RBC_binary', 'WBC_binary',
       'esterase_binary', 'Biopsy_HR', 'Biopsy_HW', 'Biopsy_HK', 'hr_A1',
       'hr_proba', 'hw_A2', 'hw_proba', 'hk_A3', 'hk_proba'],
      dtype='object')
Index(['rid', 'id', 'set', 'age', 'sex', 'HTN', 'DM', 'plt', 'tpro', 'alb',
       'chol', 'bu

##### col 데이터셋에서 사용된 변수만 사용하기, 'Biopsy_HR', 'Biopsy_HW', 'Biopsy_HK' 는 각 3명이 정답내놓은거(target)  그 뒤는 예측값 및 확률이라 제거하고 사용
- set변수는 train, validation, test 나눈거 group으로 구분준거라 없애고 해도 됨

In [4]:
# 데이터 셋 간 변수 차이
print(list(set(train.columns)^set(col.columns))[:7])
print(list(set(train.columns)^set(col.columns))[7:])

['병리작성일', 'id', '서식키', '연구등록번호', '병리진단', '병리번호', 'WBC_re']
['set', 'Tchol', 'RBC_binary', 'rid', 'chol', '(실명)등록번호', 'RBC_re']


In [19]:
print(train[['WBC','RBC','WBC_re','RBC_re']])
print(valid[['WBC','RBC','WBC_re','RBC_re']])
test[['WBC','RBC','WBC_re','RBC_re']]

              WBC             RBC  WBC_re  RBC_re
0      6-10(33.4)     10-20(79.0)    33.4    79.0
1       3-5(31.2)      6-10(45.8)    31.2    45.8
2      6-10(46.2)       3-5(28.9)    46.2    28.9
3        0-2(7.2)  None seen(4.1)     7.2     4.1
4     10-20(97.6)     Many(394.9)    97.6   394.9
...           ...             ...     ...     ...
6166            0               0     0.0     0.0
6167           30              30    30.0    30.0
6168            0              20     0.0    20.0
6169           10              30    10.0    30.0
6170            0              20     0.0    20.0

[6171 rows x 4 columns]
                WBC             RBC  WBC_re  RBC_re
0    None seen(2.9)      6-10(56.2)     2.9    56.2
1       10-20(64.7)       3-5(24.4)    64.7    24.4
2          0-2(7.9)       0-2(13.3)     7.9    13.3
3         3-5(29.2)       0-2(13.4)    29.2    13.4
4          0-2(5.7)       0-2(15.9)     5.7    15.9
..              ...             ...     ...     ...
406       0

,WBC,RBC,WBC_re,RBC_re
0,Many(162.0),0-2(5.8),162.0,5.8
1,0-2(11.4),Many(6915.1),11.4,6915.1
2,NaN,NaN,NaN,NaN
3,None seen(3.1),0-2(13.0),3.1,13.0
4,0-2(14.3),0-2(15.2),14.3,15.2
...,...,...,...,...
1641,0-2(10.0),0-2(10.0),10.0,10.0
1642,0-2(10.0),0-2(10.0),10.0,10.0
1643,0-2(10.0),0-2(10.0),10.0,10.0
1644,21~30,0-2(10.0),NaN,10.0


In [12]:
tr = train.loc[:, ~train.columns.isin(['rid','id','set','chol','hr_A1',
       'hr_proba', 'hw_A2', 'hw_proba', 'hk_A3', 'hk_proba','WBC','RBC'])].copy()
val = valid.loc[:, ~valid.columns.isin(['rid','id','set','chol','hr_A1',
       'hr_proba', 'hw_A2', 'hw_proba', 'hk_A3', 'hk_proba','WBC','RBC'])].copy()
te = test.loc[:, ~test.columns.isin(['rid','id','set','chol','hr_A1',
       'hr_proba', 'hw_A2', 'hw_proba', 'hk_A3', 'hk_proba','WBC','RBC'])].copy()

In [13]:
print(list(set(tr.columns)^set(col.columns))[:7])
print(list(set(tr.columns)^set(col.columns))[7:])

print(list(set(val.columns)^set(col.columns))[:7])
print(list(set(val.columns)^set(col.columns))[7:])

print(list(set(te.columns)^set(col.columns))[:7])
print(list(set(te.columns)^set(col.columns))[7:])


['병리작성일', '서식키', 'hr_proba', 'RBC', '연구등록번호', '병리진단', '병리번호']
['hw_A2', 'WBC_re', 'Tchol', 'WBC', 'RBC_binary', 'hk_proba', 'hr_A1', 'hw_proba', '(실명)등록번호', 'hk_A3', 'RBC_re']
['병리작성일', '서식키', 'hr_proba', 'RBC', '연구등록번호', '병리진단', '병리번호']
['hw_A2', 'WBC_re', 'Tchol', 'WBC', 'RBC_binary', 'hk_proba', 'hr_A1', 'hw_proba', '(실명)등록번호', 'hk_A3', 'RBC_re']
['병리작성일', '서식키', 'hr_proba', 'RBC', '연구등록번호', '병리진단', '병리번호']
['hw_A2', 'WBC_re', 'Tchol', 'WBC', 'RBC_binary', 'hk_proba', 'hr_A1', 'hw_proba', '(실명)등록번호', 'hk_A3', 'RBC_re']


In [14]:
print(tr.shape)
print(val.shape)
print(te.shape)

(6171, 26)
(411, 26)
(1646, 26)


In [15]:
# 열 제외 방법
train.loc[:, (train.columns!='rid')&(train.columns!='id')].head(1)
train.loc[:, ~train.columns.isin(['rid','id'])].head(1)
train.loc[:, [i for i in list(train.columns) if i not in ['rid','id']]].head(1)

,set,age,sex,HTN,DM,plt,tpro,alb,chol,bun,...,esterase_binary,Biopsy_HR,Biopsy_HW,Biopsy_HK,hr_A1,hr_proba,hw_A2,hw_proba,hk_A3,hk_proba
0,1,48.769863,2,0,0,169.0,6.1,3.5,205,24.3,...,1,1,1,1,1,0.243998,1,0.233381,1,0.11308


### 선생님별로 데이터셋 나누기

In [16]:
# train set split
hr_tr = tr.drop(['Biopsy_HW','Biopsy_HK'], axis=1).copy()
hw_tr = tr.drop(['Biopsy_HR','Biopsy_HK'], axis=1).copy()
hk_tr = tr.drop(['Biopsy_HR','Biopsy_HW'], axis=1).copy()

print(hr_tr.columns);print(hw_tr.columns);print(hk_tr.columns)
print(hr_tr.shape);print(hw_tr.shape);print(hk_tr.shape)

print('                                                   ======validation=======                                                       ')
# validation split
hr_val = val.drop(['Biopsy_HW','Biopsy_HK'], axis=1).copy()
hw_val = val.drop(['Biopsy_HR','Biopsy_HK'], axis=1).copy()
hk_val = val.drop(['Biopsy_HR','Biopsy_HW'], axis=1).copy()

print(hr_val.columns);print(hw_val.columns);print(hk_val.columns)
print(hr_val.shape);print(hw_val.shape);print(hk_val.shape)

print('                                                      ======test=======                                                              ')
# test split
hr_te = te.drop(['Biopsy_HW','Biopsy_HK'], axis=1).copy()
hw_te = te.drop(['Biopsy_HR','Biopsy_HK'], axis=1).copy()
hk_te = te.drop(['Biopsy_HR','Biopsy_HW'], axis=1).copy()

print(hr_te.columns);print(hw_te.columns);print(hk_te.columns)
print(hr_te.shape);print(hw_te.shape);print(hk_te.shape)

Index(['age', 'sex', 'HTN', 'DM', 'plt', 'tpro', 'alb', 'bun', 'cr', 'eGFR',
       'blood_dip', 'protein_dip', 'acr_urine', 'pcr_urine', 'glu',
       'glu_urine', 'esterase_urine', 'dysmorphism', 'RBC_re', 'WBC_re',
       'RBC_binary', 'WBC_binary', 'esterase_binary', 'Biopsy_HR'],
      dtype='object')
Index(['age', 'sex', 'HTN', 'DM', 'plt', 'tpro', 'alb', 'bun', 'cr', 'eGFR',
       'blood_dip', 'protein_dip', 'acr_urine', 'pcr_urine', 'glu',
       'glu_urine', 'esterase_urine', 'dysmorphism', 'RBC_re', 'WBC_re',
       'RBC_binary', 'WBC_binary', 'esterase_binary', 'Biopsy_HW'],
      dtype='object')
Index(['age', 'sex', 'HTN', 'DM', 'plt', 'tpro', 'alb', 'bun', 'cr', 'eGFR',
       'blood_dip', 'protein_dip', 'acr_urine', 'pcr_urine', 'glu',
       'glu_urine', 'esterase_urine', 'dysmorphism', 'RBC_re', 'WBC_re',
       'RBC_binary', 'WBC_binary', 'esterase_binary', 'Biopsy_HK'],
      dtype='object')
(6171, 24)
(6171, 24)
(6171, 24)
                                           

#### 데이터 셋 내보내기

In [17]:
hr_tr.to_csv('../data/hr_tr.csv', index=False)
hw_tr.to_csv('../data/hw_tr.csv', index=False)
hk_tr.to_csv('../data/hk_tr.csv', index=False)

hr_val.to_csv('../data/hr_val.csv', index=False )
hw_val.to_csv('../data/hw_val.csv', index=False)
hk_val.to_csv('../data/hk_val.csv', index=False )

hr_te.to_csv('../data/hr_te.csv', index=False )
hw_te.to_csv('../data/hw_te.csv', index=False )
hk_te.to_csv('../data/hk_te.csv', index=False )

In [18]:
xgb = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=4, random_state = 2023,
                    tree_method='gpu_hist', gpu_id=0)